In [2]:
# Import libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
import glob
import pickle
import re
from difflib import SequenceMatcher
from dateutil import parser

In [3]:
covers = pickle.load(open("data/covers_algo_results.p", "rb"))

In order to manipulate dates, we import the parser from the dateutil python package, and convert the date column in string. Some date containing only the year are floats so they are converted to a string of the form "(year).0", thus we also need to remove all the ".0"

In [4]:
def multiple_originals(covers, col):
    groupbyclique = covers.groupby('clique_id')

    multi_originals = []
    for c, group in groupbyclique:
        nb_original = len(group[group[col] == "original"])
        if nb_original > 1:
            multi_originals.append(c)
    return multi_originals

In [5]:
dup_originals = multiple_originals(covers, 'second_algo_result')

In [6]:
len(dup_originals)

380

In [7]:
def get_val(covers, track, col):
    index = covers[covers['trackID'] == track].index[0]
    return covers.get_value(index, col)

In [8]:
covers['date'] = covers['date'].map(lambda x : str(x).replace(".0", "")) 

covers['country'].replace("Missing", np.nan, inplace=True)
covers['country'].replace("Unavailable", np.nan, inplace=True)

covers["final_original"] = covers['trackID'].\
map(lambda x : "nan" if get_val(covers, x, 'clique_id') in dup_originals else get_val(covers, x, 'second_algo_result'))


Now, we define the merge_algo function that, given a dataframe of tracks within a clique will return the original. The original returned is the one with the oldest date of release. If the oldest year appears mutliple times, we will check if we can find the original from the first algorithm. By doing some tests, there are only 73 cliques for which we cannot determine the original by merging the two algorithms. We will then manually define the original for these cliques.

In [15]:
def find_original(tracks):
    default = parser.parse("December 31, 2017")
    dates = tracks['date'].map(lambda x : parser.parse(str(x), default=default)).tolist()
    oldest = dates.index(min(dates))
    count = dates.count(min(dates))
    index = tracks.iloc[[oldest]].index[0]
    clique_id = tracks.get_value(index, 'clique_id')
    if count > 1:
        date = tracks.get_value(index, "date")
        tracks_date = tracks[tracks['date'] == date]
        first_algo = tracks[tracks['first_algo_result'] == "original"]
        if first_algo.empty:
            null_nb = tracks_date.isnull().sum(axis=1).tolist()
            more_complete = null_nb.index(min(null_nb))
            index = tracks.iloc[[more_complete]].index[0]
        else:
            index = first_algo.head(1).index[0]
    return index
    

In [16]:
def keep_one_original(covers, clique_id):
    clique = covers[covers['clique_id'] == clique_id]
    tracks = clique[clique['second_algo_result'] == 'original']
    original_index = find_original(tracks)
    covers.set_value(original_index, 'final_original', "original")

In [17]:
for c in dup_originals:
    keep_one_original(covers, c)

In [20]:
covers['final_original'].replace('nan', 'cover', inplace=True)

In [24]:
check_dup = multiple_originals(covers, 'final_original')
len(check_dup)

0

In [25]:
pickle.dump(covers, open("data/covers_merge_algo.p", "wb"))

In [76]:
cover_songs = covers_merge[covers_merge['final_original'] != "original"]

In [88]:
countcovers = cover_songs.groupby('year').count()[['trackID']]
countcovers.columns = [['count']]

In [102]:
countcovers.reset_index(inplace = True)

In [272]:
countcovers[countcovers['year'].isin(["2016","2017"])]

,year,count


In [285]:
def print_values(y1):
    i1 = countoriginals[countoriginals['year'] == str(y1)].index[0]
    v1 = countoriginals.get_value(i1, 'count')
    y2 = y1+1
    #v1 = 0
    i2 = countoriginals[countoriginals['year'] == str(y2)].index[0]
    v2 = countoriginals.get_value(i2, 'count')
    height = v1+v2
    
    y = 329 - height
    print("height = " + str(height) + "\ny = " + str(y))

In [332]:
print_values(2008)

height = 23
y = 306


In [283]:
originals = covers_merge[covers_merge['final_original'] == "original"]
countoriginals = originals.groupby('year').count()[['trackID']]
countoriginals.columns = [['count']]
countoriginals.reset_index(inplace=True)

In [284]:
countoriginals

,year,count
0,,17
1,1911,1
2,1923,1
3,1925,4
4,1926,2
5,1927,8
6,1928,13
7,1929,14
8,1930,11
9,1931,14


In [14]:
groupbyclique